Fylype Wase Cabral Lira

Mardoqueu Arruda Pimentel

Davi Simite Damasceno

In [1]:
import cv2
from matplotlib import pyplot as plt
import json
import os
from helper import (
    read_img, 
    show_img, 
    histogram, 
    cossine_similarity, 
    distance_two_frames, 
    get_total_frames, 
    get_fps, 
    get_frames_from_video,
    get_seconds_from_frame_number,
    change_scene,
    accuracy,
    generate_index_frames_to_compare,
    divide_3_partitions,
    divide_5_by_5,
    save_image
)

%matplotlib inline

In [2]:
with open("seconds_change_scene_videos.json", "r") as f:
    videos_metadata = json.load(f)

Os arquivos necessários para esse trabalho estão no Google Drive seguinda esse link:
https://drive.google.com/drive/folders/1hPN3_JDsjRky1UAt-ptrQRaUpsAnH7FY?usp=sharing

Para detectar trocar de cenas consideramos a diferença entre 2 frames. Esta diferença é dada pela similaridade entre os frames, essa similiridade é dada pela similaridade do cosseno para as 3 bandas. Se a similaridade tiver abaixo de um limar escolhido então é considerado como troca de cena. Esse limiar foi escolhido atráves de alguns experimentos feitos pelo trio.

Consideramos janela de 5 frames para acelerar o processamento.

Coletamos 10 videos com o tempo estipulado no enunciado do trabalho que estão link do Drive disponibilizado acima. Para cada video calculamos sua acurácia dos acertos em comparação com a detecção manual de cena.

Abaixo há a detecção utilizando o histograma global com o limiar de 0.75. Os quadros chaves estão disponíveis em `video/global_histogram`

In [4]:
for video_metadata in videos_metadata:
    file_name = video_metadata.get("video")
    change_scenes_seconds_true = video_metadata.get("cortes")
    video = cv2.VideoCapture(f'videos/{file_name}.mp4')
    
    window = 5
    frames = get_frames_from_video(video, window=window)
    
    fps = get_fps(video)
    change_scenes_seconds_detect = []
    
    path_file = f"images/global_histogram/{file_name}"
    os.makedirs(path_file, exist_ok=True)

    for index_frame in range(1, len(frames)):
        if change_scene(frames[index_frame], frames[index_frame-1], threshold=0.75):
            second_detect = get_seconds_from_frame_number(index_frame * window, fps)

            if second_detect not in change_scenes_seconds_detect:
                change_scenes_seconds_detect.append(second_detect)
                save_image(frames[index_frame], f"{path_file}/quadro_chave_{index_frame}.jpg")
                
    accuracy_change_scene = accuracy(change_scenes_seconds_true, change_scenes_seconds_detect)
    
    print(f"Video {file_name}. Accuracy: {accuracy_change_scene}")

Video 1 Minute Video - Doggie. Accuracy: 0.625
Video 6 AM _ Film Riot - One Minute Short Contest. Accuracy: 0.10810810810810811
Video BABY DRIVER - 6-Minute Opening Clip. Accuracy: 0.02252252252252252
Video Coldplay - Paradise (Official Video). Accuracy: 0.02857142857142857
Video Cowboy Bebop – Opening Theme – Tank!. Accuracy: 0.19298245614035087
Video First Aid Fail - The Office US. Accuracy: 0.3939393939393939
Video Jornal Nacional - Matéria sobre Marketing Digital. Accuracy: 0.782608695652174
Video Juntos a magia acontece. Accuracy: 0.034482758620689655
Video Marvel Studios_ Avengers_ Endgame - Official Trailer. Accuracy: 0.171875
Video videoplayback. Accuracy: 0.5194805194805194


Abaixo há a detecção utilizando o histograma com 3 partições com o limiar de 0.75. Os quadros chaves estão disponíveis em `video/3_partitions`.
Para cada partição verificamos, com a mesma politica de similaridade comentada acima, se há a existência de 2 subframes diferentes. Em caso de confirmação positiva de 2 subframes diferentes então consideramos que há um troca de cena.

Consideramos janela de 5 frames para acelerar o processamento.

In [6]:
for video_metadata in videos_metadata:
    file_name = video_metadata.get("video")
    change_scenes_seconds_true = video_metadata.get("cortes")
    video = cv2.VideoCapture(f'videos/{file_name}.mp4')
    
    window = 5
    frames = get_frames_from_video(video, window=window)
    
    fps = get_fps(video)
    change_scenes_seconds_detect = []
    
    path_file = f"images/3_partitions/{file_name}"
    os.makedirs(path_file, exist_ok=True)

    for index_frame in range(1, len(frames)):
        partitions_frame_1 = divide_3_partitions(frames[index_frame])
        partitions_frame_2 = divide_3_partitions(frames[index_frame-1])
        change_scene_partition = [
            change_scene(part_frame_1, part_frame_2, threshold=0.75)
            for part_frame_1, part_frame_2
            in zip(partitions_frame_1, partitions_frame_2)
        ]

        if True in change_scene_partition:
            second_detect = get_seconds_from_frame_number(index_frame * window, fps)

            if second_detect not in change_scenes_seconds_detect:
                change_scenes_seconds_detect.append(second_detect)
                save_image(frames[index_frame], f"{path_file}/quadro_chave_{index_frame}.jpg")
                
    accuracy_change_scene = accuracy(change_scenes_seconds_true, change_scenes_seconds_detect)
    
    print(f"Video {file_name}. Accuracy: {accuracy_change_scene}")

Video 1 Minute Video - Doggie. Accuracy: 0.8125
Video 6 AM _ Film Riot - One Minute Short Contest. Accuracy: 0.1891891891891892
Video BABY DRIVER - 6-Minute Opening Clip. Accuracy: 0.04954954954954955
Video Coldplay - Paradise (Official Video). Accuracy: 0.15714285714285714
Video Cowboy Bebop – Opening Theme – Tank!. Accuracy: 0.24561403508771928
Video First Aid Fail - The Office US. Accuracy: 0.7272727272727273
Video Jornal Nacional - Matéria sobre Marketing Digital. Accuracy: 0.9565217391304348
Video Juntos a magia acontece. Accuracy: 0.034482758620689655
Video Marvel Studios_ Avengers_ Endgame - Official Trailer. Accuracy: 0.203125
Video videoplayback. Accuracy: 0.7402597402597403


Abaixo há a detecção utilizando o histograma com partiticionamento em grid 5 x 5 com o limiar de 0.65. Os quadros chaves estão disponíveis em `video/matrix_5_x_5`.
Para cada partição verificamos, com a mesma politica de similaridade comentada acima, se há a existência de 2 subframes diferentes. Em caso de confirmação positiva de 2 subframes diferentes então consideramos que há um troca de cena.

Consideramos janela de 5 frames para acelerar o processamento.

In [3]:
for video_metadata in videos_metadata:
    file_name = video_metadata.get("video")
    change_scenes_seconds_true = video_metadata.get("cortes")
    video = cv2.VideoCapture(f'videos/{file_name}.mp4')
    
    window = 5
    frames = get_frames_from_video(video, window=window)
    
    fps = get_fps(video)
    change_scenes_seconds_detect = []
    
    path_file = f"images/matrix_5_x_5/{file_name}"
    os.makedirs(path_file, exist_ok=True)

    for index_frame in range(1, len(frames)):
        partitions_frame_1 = divide_5_by_5(frames[index_frame])
        partitions_frame_2 = divide_5_by_5(frames[index_frame-1])
        change_scene_partition = [
            change_scene(part_frame_1, part_frame_2, threshold=0.65)
            for part_frame_1, part_frame_2
            in zip(partitions_frame_1, partitions_frame_2)
        ]

        if True in change_scene_partition:
            second_detect = get_seconds_from_frame_number(index_frame * window, fps)

            if second_detect not in change_scenes_seconds_detect:
                change_scenes_seconds_detect.append(second_detect)
                save_image(frames[index_frame], f"{path_file}/quadro_chave_{index_frame}.jpg")
                
    accuracy_change_scene = accuracy(change_scenes_seconds_true, change_scenes_seconds_detect)
    
    print(f"Video {file_name}. Accuracy: {accuracy_change_scene}")

Video 1 Minute Video - Doggie. Accuracy: 1.0
Video 6 AM _ Film Riot - One Minute Short Contest. Accuracy: 0.918918918918919
Video BABY DRIVER - 6-Minute Opening Clip. Accuracy: 0.9819819819819819
Video Coldplay - Paradise (Official Video). Accuracy: 0.9428571428571428
Video Cowboy Bebop – Opening Theme – Tank!. Accuracy: 0.5789473684210527
Video First Aid Fail - The Office US. Accuracy: 0.7575757575757576
Video Jornal Nacional - Matéria sobre Marketing Digital. Accuracy: 1.0
Video Juntos a magia acontece. Accuracy: 0.8620689655172413
Video Marvel Studios_ Avengers_ Endgame - Official Trailer. Accuracy: 0.96875
Video videoplayback. Accuracy: 0.8311688311688312
